In [52]:
from selenium import webdriver
from time import sleep
import wget
import pymysql
import re
import os

browser = webdriver.Firefox()
browser.get('https://www.gutenberg.org/ebooks/search/?query=sherlock+holmes+conan')
browser.implicitly_wait(3)

books = browser.find_elements_by_class_name('booklink')

for book in range(0, 25):
    books[book].click()
    html_link = browser.find_elements_by_class_name('link')[0]
    html_link.click()

    sleep(2)
    wget.download(browser.current_url, out='Books')
    sleep(2)

    browser.get('https://www.gutenberg.org/ebooks/search/?query=sherlock+holmes+conan')
    books = browser.find_elements_by_class_name('booklink')

browser.close()

In [123]:
cnx = pymysql.connect(user='dev', password='ax2',host='127.0.0.1',port=3306,db='test')
query = 'INSERT INTO sections (pound, start, end, section, amount, filename) VALUES (%(pound)s, %(start)s, %(end)s, %(section)s, %(amount)s, %(filename)s)'

pound_reg = re.compile(r'(£)(\s|\n)*(\d+,?\d*)')

for filename in os.listdir('Books'):
    with open('Books/' + filename, 'r', encoding="ISO-8859-1") as fo:
        text = fo.read()
        for m in pound_reg.finditer(text):
            with cnx.cursor() as cursor:
                start = text.rfind('<p', 0, m.start())
                stop = text.find('</p>', m.start())
                section = {
                    'pound' : m.start(),
                    'start' : start,
                    'end' : stop,
                    'section' : text[start:stop],
                    'amount' : int(m.group(3).replace(',', '')),
                    'filename' : filename
                }
                cursor.execute(query, section)

cnx.commit()
cnx.close()

In [124]:
cnx = pymysql.connect(user='dev', password='ax2',host='127.0.0.1',port=3306,db='test')
query = 'SELECT COUNT(*) FROM sections'

with cnx.cursor() as cursor:
    cursor.execute(query)
    print('There are {} £ in the first 25 books'.format(next(cursor.__iter__())[0]))

cnx.close()

There are 43 £ in the first 25 books


In [125]:
cnx = pymysql.connect(user='dev', password='ax2',host='127.0.0.1',port=3306,db='test')
query = 'SELECT amount, section FROM sections ORDER BY amount DESC LIMIT 0, 1'

with cnx.cursor() as cursor:
    cursor.execute(query)
    biggest = next(cursor.__iter__())
    print('Biggest amount is {} and the section it\'s in is: \n\n{}'.format(biggest[0], biggest[1]))

cnx.close()

Biggest amount is 50000 and the section it's in is: 

<p>
&ldquo;&lsquo;It is absolutely essential to me,&rsquo; said he, &lsquo;that I
should have Â£ 50,000 at once. I could, of course, borrow so trifling a sum ten
times over from my friends, but I much prefer to make it a matter of business
and to carry out that business myself. In my position you can readily
understand that it is unwise to place one&rsquo;s self under obligations.&rsquo;

